In [1]:
import numpy as np
import gymnasium as gym
from tqdm import tqdm



env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", 
               is_slippery=False, render_mode="human")

MAX_ITERATIONS = 1000
 
env.reset()
env.render()
for i in range(MAX_ITERATIONS):
    random_action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(random_action)
    env.render()
    if terminated or truncated:
        break

In [32]:
num_states = env.observation_space.n
print(f'num_states: {num_states}')

num_actions = env.action_space.n
print(f'num_actions: {num_actions}')

state_action_value_function = np.zeros((num_states, num_actions))
print(f'value_function: {state_action_value_function} |  shape: {state_action_value_function.shape}')

num_episodes = 10
value_function_tracker = np.zeros((num_states, num_actions, num_episodes))
print(f'value_function_tracker: \n{value_function_tracker} |  shape: {value_function_tracker.shape}')

state_value_function = np.zeros(num_states)
state_value_function_tracker = np.zeros((num_states, num_episodes))


# Define the number of steps for the n-step TD learning algorithm
n_step = 3


# Define the discount factor
discount_factor = 0.99

discount_factors = np.logspace(0, n_step + 1, num=n_step+1, base=discount_factor, endpoint=False)
print(f'discount_factor: {discount_factors} |  shape: {discount_factors.shape}')

num_states: 16
num_actions: 4
value_function: [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] |  shape: (16, 4)
value_function_tracker: 
[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0

In [33]:
def policy(state, q_function):
    # Assume `state` is the current state.
    # Assume `q_function` is state action value function.

    # Find the action that leads to the state with the highest value.
    max_value = np.max(q_function[state])
    best_actions = np.where(q_function[state] == max_value)[0]

    # Randomly choose one of the best actions.
    action = np.random.choice(best_actions)

    return action

def policy(state=None):
    """
    A simple policy function for the FrozenLake environment.
    This policy always selects a random action.

    Parameters:
    state: The current state of the environment.

    Returns:
    action: The action to be taken in the current state.
    """
    action = env.action_space.sample()  # Select a random action
    return action



In [34]:
action_to_label = {0: "Move left",
                   1: "Move down",
                   2: "Move right",
                   3: "Move up",}

In [35]:
# Loop though each episode
for episode in tqdm(range(2), leave=False):
    print(f'episode: {episode}')
    
    current_state, terminated_truncated, trajectories = env.reset(), False, []
    current_state = current_state[0]
    next_state = None 

    # Episode loop
    while not terminated or trajectories is not None:
        # Here, we area popping the first element of the path.
        trajectories = trajectories[1:]

        while not terminated_truncated and len(trajectories) < n_step:
            print(f'trajectories length: {len(trajectories)}')
            # No action is taken yet
            print(f'current_state: {current_state}')

            # Here, we can choose to either use the policy or the state-action value function
            # to get the action to take.
            action = policy()
            print(f'action: {action_to_label[action]}')

            # Once the action was taken by the agent using the policy, 
            # we can get the next state, reward, termination status, etc.
            observation, reward, terminated, truncated, info = env.step(action)

            # Inclusive or for terminated and truncated
            terminated_truncated = terminated or truncated

            # Update the next state
            next_state = observation

            # Print the results of the action taken.
            print(f'observation (new state): {observation}')
            print(f'reward: {reward}')
            print(f'terminated: {terminated}')
            print(f'truncated: {truncated}')

            # Organize the experience in a structured  way
            experience = {
                'state': current_state,
                'action': action,
                'reward': reward,
                'next_state': next_state,
                'terminated_truncated': terminated_truncated
            }

            # Append the experience in the trajectories variable
            trajectories.append(experience)

            # Update the current state to the new state
            current_state = next_state
            print('-----' * 10)

            # Check if the episode is over (terminated or truncated)
            if terminated_truncated:
                print('Episode is over'.upper())
                print('-----' * 10)
                break

        print("OUTSIDE THE WHILE LOOP")
        # Prepare variables for tD target ad error
        # num_experiences could be 'n_steps' but is could also be a smaller number
        # if a terminal state is in the 'path'
        num_experiences = len(trajectories)
        print(f'num_experiences: {num_experiences}')

        estimated_state = trajectories[0]['state']
        print(f'estimated_state: {estimated_state}')

        estimated_action = trajectories[0]['action']
        print(f'estimated_action: {estimated_action}')
        
        rewards = [trajectory['reward'] for  trajectory in trajectories ]
        print(f'rewards: {rewards}'	)

        partial_return = discount_factors[:num_experiences] * rewards
        print(f'partial_return: {partial_return}')

        bootstrap_value = discount_factors[-1] * state_action_value_function[next_state] * (not terminated)
        print(f'bootstrap_value: {bootstrap_value}')

        # n_step_td_target is  the sum of the partial return and the bootstrap value
        n_step_td_target = np.sum(np.append(partial_return, bootstrap_value))

        # the n_step_td_error is the difference between the n_step_td_target and the estimated state-action value function
        # n_step_td_error = n_step_td_target - state_action_value_function[]

        print('-----' * 10)



  0%|          | 0/2 [00:00<?, ?it/s]

episode: 0
trajectories length: 0
current_state: 0
action: Move right
observation (new state): 1
reward: 0.0
terminated: False
truncated: False
--------------------------------------------------
trajectories length: 1
current_state: 1
action: Move left
observation (new state): 0
reward: 0.0
terminated: False
truncated: False
--------------------------------------------------
trajectories length: 2
current_state: 0
action: Move up
observation (new state): 0
reward: 0.0
terminated: False
truncated: False
--------------------------------------------------
OUTSIDE THE WHILE LOOP
num_experiences: 3
estimated_state: 0
estimated_action: 2
rewards: [0.0, 0.0, 0.0]
partial_return: [0. 0. 0.]
bootstrap_value: [0. 0. 0. 0.]
--------------------------------------------------
trajectories length: 2
current_state: 0
action: Move right
observation (new state): 1
reward: 0.0
terminated: False
truncated: False
--------------------------------------------------
OUTSIDE THE WHILE LOOP
num_experiences: 3


observation (new state): 7
reward: 0.0
terminated: True
truncated: False
--------------------------------------------------
EPISODE IS OVER
--------------------------------------------------
OUTSIDE THE WHILE LOOP
num_experiences: 3
estimated_state: 1
estimated_action: 2
rewards: [0.0, 0.0, 0.0]
partial_return: [0. 0. 0.]
bootstrap_value: [0. 0. 0. 0.]
--------------------------------------------------
OUTSIDE THE WHILE LOOP
num_experiences: 2
estimated_state: 2
estimated_action: 2
rewards: [0.0, 0.0]
partial_return: [0. 0.]
bootstrap_value: [0. 0. 0. 0.]
--------------------------------------------------
OUTSIDE THE WHILE LOOP
num_experiences: 1
estimated_state: 3
estimated_action: 1
rewards: [0.0]
partial_return: [0.]
bootstrap_value: [0. 0. 0. 0.]
--------------------------------------------------
OUTSIDE THE WHILE LOOP
num_experiences: 0


IndexError: list index out of range

In [ ]:
partial_return

In [ ]:
rewards

In [ ]:
trajectories

In [ ]:
trajectory = trajectory[1:]
print(f'Trajectory:')
for point in trajectory:
    print(f'\t{point}')

In [ ]:
# Find the action that leads to the state with the highest value.
max_value = np.max(state_action_value_function[current_state])
best_actions = np.where(state_action_value_function[current_state] == max_value)[0]

# Randomly choose one of the best actions.
action = np.random.choice(best_actions)
action

In [ ]:
action = np.random.choice(best_actions)

In [15]:
import random

def policy(state=None):
    return random.randint(0, 3)

policy()

0

In [5]:
%load_ext autoreload
%autoreload 2

In [25]:
from src.n_step_td import n_step_td


# Set the parameters for n-step TD learning
discount_factor = 1.0
initial_learning_rate = 0.5
min_learning_rate = 0.01
learning_rate_decay_ratio = 0.5
n_step = 3
num_episodes = 500

# Run n-step TD learning
value_function, value_function_tracker = n_step_td(policy, env, discount_factor, 
                                                   initial_learning_rate,
                                                   min_learning_rate, learning_rate_decay_ratio,
                                                   n_step, num_episodes)

# Print the final value function
print("Value Function:")
print(value_function)

  0%|          | 1/500 [00:00<04:11,  1.98it/s]

0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 4
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 1
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 4
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 4


  0%|          | 2/500 [00:03<15:10,  1.83s/it]

0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 1
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 1
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 4
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 1
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 1
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0


  1%|          | 3/500 [00:05<17:42,  2.14s/it]

...

  6%|▌         | 28/500 [01:02<12:04,  1.53s/it]

0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 4


  6%|▌         | 29/500 [01:03<10:11,  1.30s/it]

0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0
0.0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 4


KeyboardInterrupt: 

In [1]:
def simple_decorator(function):
    print('We are about to call "{}"'.format(function.__name__))
    return function


@simple_decorator
def simple_hello():
    print("Hello from simple function!")

We are about to call "simple_hello"


In [2]:
(1 - λ) * [ λ^(1-1) * G2:2+1 + λ^(2-1) * G2:2+2 ] + λ^(5-2-1) * G0:5
(1 - λ) * [ λ^(1-1) * G2:2+1 + λ^(2-1) * G2:2+2 ] + λ^(5-2-1) * G2:5

Hello from simple function!
